***GENERATED CODE FOR 1609211routingmodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternalOrderIDRef', 'transformation_label': 'String Indexer'}], 'feature': 'InternalOrderIDRef', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'DEMO Order 5-10', 'max': 'clone-92998', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternalOrderIDRef'}, {'feature_label': 'InternalOrderIDRef', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternalOrderIDRef')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderDate', 'transformation_label': 'String Indexer'}], 'feature': 'OrderDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '2018-01-10 17:41', 'max': '2019-12-11 17:42', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderDate'}, {'feature_label': 'OrderDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerFirstName', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerFirstName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ADDIE', 'max': 'ZENOBIA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerFirstName'}, {'feature_label': 'CustomerFirstName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerFirstName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerLastName', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerLastName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AHMAD', 'max': 'doe', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerLastName'}, {'feature_label': 'CustomerLastName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerLastName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerPhone', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerPhone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5566522017.29', 'stddev': '2239504728.76', 'min': '2058633922', 'max': '9727929102', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerPhone'}, {'feature_label': 'CustomerPhone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerPhone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductName', 'transformation_label': 'String Indexer'}], 'feature': 'ProductName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '"JCP Frigidaire ENERGY STAR® 18"" Built-In Dishwasher"', 'max': 'rosetti riveting seams hobo bag', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductName'}, {'feature_label': 'ProductName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAddress', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAddress', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 Harbor Ln', 'max': 'Lot Q (17) Ponderosa Dr', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAddress'}, {'feature_label': 'CustomerAddress', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerAddress')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerCity', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Abilene', 'max': 'Wylie', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerCity'}, {'feature_label': 'CustomerCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerState', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'NY', 'max': 'WA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerState'}, {'feature_label': 'CustomerState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerCountry', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerCountry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'United States', 'max': 'United States', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerCountry'}, {'feature_label': 'CustomerCountry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerCountry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ShippingMethod', 'transformation_label': 'String Indexer'}], 'feature': 'ShippingMethod', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' 2 Hours Delivery', 'max': 'Store Pickup', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ShippingMethod'}, {'feature_label': 'ShippingMethod', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ShippingMethod')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Name', 'transformation_label': 'String Indexer'}], 'feature': 'Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '"JCP Frigidaire ENERGY STAR® 18"""" Built-In Dishwasher"', 'max': 'sunday riley tidal + auto correct set', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Name'}, {'feature_label': 'Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Name')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SKU', 'threshold': 2108992113.572, 'transformation_label': 'Binarizer'}], 'feature': 'SKU', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '395202143258.79', 'stddev': '366854829044.44', 'min': '333333.0', 'max': '8.9E11', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'SKU'}, {'feature_label': 'SKU', 'threshold': 2108992113.572, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('SKU')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RetailerItemId', 'transformation_label': 'String Indexer'}], 'feature': 'RetailerItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '398914861.26', 'stddev': '282687658.8', 'min': '10389826', 'max': 'PKN103', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RetailerItemId'}, {'feature_label': 'RetailerItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RetailerItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DimensionsUnit', 'transformation_label': 'String Indexer'}], 'feature': 'DimensionsUnit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Inches              ', 'max': 'inches              ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DimensionsUnit'}, {'feature_label': 'DimensionsUnit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DimensionsUnit')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'QuantityUnit', 'transformation_label': 'String Indexer'}], 'feature': 'QuantityUnit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'PCS                 ', 'max': 'PCS                 ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'QuantityUnit'}, {'feature_label': 'QuantityUnit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('QuantityUnit')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'weight_unit', 'transformation_label': 'String Indexer'}], 'feature': 'weight_unit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'LBS', 'max': 'LBS', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'weight_unit'}, {'feature_label': 'weight_unit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('weight_unit')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationType', 'transformation_label': 'String Indexer'}], 'feature': 'LocationType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'DC', 'max': 'Store', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationType'}, {'feature_label': 'LocationType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StoreRefId', 'transformation_label': 'String Indexer'}], 'feature': 'StoreRefId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Store 1', 'max': 'store 29', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StoreRefId'}, {'feature_label': 'StoreRefId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StoreRefId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StoreAddress', 'transformation_label': 'String Indexer'}], 'feature': 'StoreAddress', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10500 Lackman Rd', 'max': 'Tarzan', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StoreAddress'}, {'feature_label': 'StoreAddress', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StoreAddress')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StoreCity', 'transformation_label': 'String Indexer'}], 'feature': 'StoreCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Amarillo', 'max': 'Wildorado', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StoreCity'}, {'feature_label': 'StoreCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StoreCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StoreState', 'transformation_label': 'String Indexer'}], 'feature': 'StoreState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'KS', 'max': 'TX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StoreState'}, {'feature_label': 'StoreState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StoreState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StoreCountry', 'transformation_label': 'String Indexer'}], 'feature': 'StoreCountry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'United States', 'max': 'United States', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StoreCountry'}, {'feature_label': 'StoreCountry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StoreCountry')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent, k):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()
    model = KMeans(n_clusters=k).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        clusters = stageAttributes['totalClusters']

        bkmeans_model = kmeans(
            df=spark_DF, TrainingPercent=trainingPercent, k=clusters)


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run 1609211routingmodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	routingdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1631620758415_RoutingData_140921.csv', 'filename': '1631620758415_RoutingData_140921.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/BiglynxUpdated/RoutingData/RoutingData_140921.csv', 'viewFileName': 'RoutingData_140921.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 1609211routingmodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(routingdata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "OrderId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4562081927.78", "stddev": "3096037298.4", "min": "1248882", "max": "9998845087", "missing": "0"}, "updatedLabel": "OrderId"}, {"transformationsData": [{"feature_label": "InternalOrderIDRef", "transformation_label": "String Indexer"}], "feature": "InternalOrderIDRef", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DEMO Order 5-10", "max": "clone-92998", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternalOrderIDRef"}, {"transformationsData": [{"feature_label": "OrderDate", "transformation_label": "String Indexer"}], "feature": "OrderDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "2018-01-10 17:41", "max": "2019-12-11 17:42", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetailerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "265568539.0", "stddev": "0.0", "min": "265568539", "max": "265568539", "missing": "0"}, "updatedLabel": "RetailerId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShippingId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.99", "stddev": "1.33", "min": "1", "max": "6", "missing": "0"}, "updatedLabel": "ShippingId"}, {"transformationsData": [{"feature_label": "CustomerFirstName", "transformation_label": "String Indexer"}], "feature": "CustomerFirstName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ADDIE", "max": "ZENOBIA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerFirstName"}, {"transformationsData": [{"feature_label": "CustomerLastName", "transformation_label": "String Indexer"}], "feature": "CustomerLastName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AHMAD", "max": "doe", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerLastName"}, {"transformationsData": [{"feature_label": "CustomerPhone", "transformation_label": "String Indexer"}], "feature": "CustomerPhone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5566522017.29", "stddev": "2239504728.76", "min": "2058633922", "max": "9727929102", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerPhone"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OrderLineItemId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "30223038405.4", "stddev": "31107071239.28", "min": "120001", "max": "97352206641", "missing": "0"}, "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"feature_label": "ProductName", "transformation_label": "String Indexer"}], "feature": "ProductName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"JCP Frigidaire ENERGY STAR\u00ae 18\"\" Built-In Dishwasher\"", "max": "rosetti riveting seams hobo bag", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductName"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Qty", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.93", "stddev": "8.31", "min": "1", "max": "154", "missing": "0"}, "updatedLabel": "Qty"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAddressId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2779532133.44", "stddev": "2761058935.07", "min": "70006", "max": "9881532264", "missing": "0"}, "updatedLabel": "CustomerAddressId"}, {"transformationsData": [{"feature_label": "CustomerAddress", "transformation_label": "String Indexer"}], "feature": "CustomerAddress", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 Harbor Ln", "max": "Lot Q (17) Ponderosa Dr", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAddress"}, {"transformationsData": [{"feature_label": "CustomerCity", "transformation_label": "String Indexer"}], "feature": "CustomerCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abilene", "max": "Wylie", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerCity"}, {"transformationsData": [{"feature_label": "CustomerState", "transformation_label": "String Indexer"}], "feature": "CustomerState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "NY", "max": "WA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerState"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "70912.99", "stddev": "19283.42", "min": "10004", "max": "98107", "missing": "0"}, "updatedLabel": "CustomerZip"}, {"transformationsData": [{"feature_label": "CustomerCountry", "transformation_label": "String Indexer"}], "feature": "CustomerCountry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "United States", "max": "United States", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerCountry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "31.88", "stddev": "2.78", "min": "26.1157675", "max": "47.6733725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-95.85", "stddev": "7.79", "min": "-122.407656", "max": "-73.77793299999999", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerLon"}, {"transformationsData": [{"feature_label": "ShippingMethod", "transformation_label": "String Indexer"}], "feature": "ShippingMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " 2 Hours Delivery", "max": "Store Pickup", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ShippingMethod"}, {"transformationsData": [{"feature_label": "Name", "transformation_label": "String Indexer"}], "feature": "Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"JCP Frigidaire ENERGY STAR\u00ae 18\"\"\"\" Built-In Dishwasher\"", "max": "sunday riley tidal + auto correct set", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Name"}, {"transformationsData": [{"feature_label": "SKU", "threshold": 2108992113.572, "transformation_label": "Binarizer"}], "feature": "SKU", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "395202143258.79", "stddev": "366854829044.44", "min": "333333.0", "max": "8.9E11", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "SKU"}, {"transformationsData": [{"feature_label": "RetailerItemId", "transformation_label": "String Indexer"}], "feature": "RetailerItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "398914861.26", "stddev": "282687658.8", "min": "10389826", "max": "PKN103", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RetailerItemId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PhysicalInventory", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "133.32", "stddev": "38.94", "min": "10", "max": "550", "missing": "0"}, "updatedLabel": "PhysicalInventory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OrderedReserved", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.55", "stddev": "1.9", "min": "0", "max": "9", "missing": "0"}, "updatedLabel": "OrderedReserved"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AvailableQty", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "130.71", "stddev": "40.75", "min": "10", "max": "550", "missing": "0"}, "updatedLabel": "AvailableQty"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "23.44", "stddev": "12.04", "min": "4.0", "max": "69.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Length"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Width", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "51.04", "stddev": "43.74", "min": "2.5", "max": "211.25", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Width"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Height", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.36", "stddev": "7.03", "min": "0.5", "max": "37.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Height"}, {"transformationsData": [{"feature_label": "DimensionsUnit", "transformation_label": "String Indexer"}], "feature": "DimensionsUnit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Inches              ", "max": "inches              ", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DimensionsUnit"}, {"transformationsData": [{"feature_label": "QuantityUnit", "transformation_label": "String Indexer"}], "feature": "QuantityUnit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "PCS                 ", "max": "PCS                 ", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "QuantityUnit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "weight", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.91", "stddev": "10.68", "min": "0.5", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "weight"}, {"transformationsData": [{"feature_label": "weight_unit", "transformation_label": "String Indexer"}], "feature": "weight_unit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "LBS", "max": "LBS", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "weight_unit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Cost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "86.93", "stddev": "92.61", "min": "9.0", "max": "500.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Cost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetailerLocationId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "166396566.58", "stddev": "301098103.3", "min": "1084493", "max": "866526434", "missing": "0"}, "updatedLabel": "RetailerLocationId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AddressId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "493269723.45", "stddev": "292590634.03", "min": "117945208", "max": "990887757", "missing": "0"}, "updatedLabel": "AddressId"}, {"transformationsData": [{"feature_label": "LocationType", "transformation_label": "String Indexer"}], "feature": "LocationType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DC", "max": "Store", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationType"}, {"transformationsData": [{"feature_label": "StoreRefId", "transformation_label": "String Indexer"}], "feature": "StoreRefId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Store 1", "max": "store 29", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StoreRefId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreAddressId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "556541229.21", "stddev": "292045439.01", "min": "117945208", "max": "990887757", "missing": "0"}, "updatedLabel": "StoreAddressId"}, {"transformationsData": [{"feature_label": "StoreAddress", "transformation_label": "String Indexer"}], "feature": "StoreAddress", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10500 Lackman Rd", "max": "Tarzan", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StoreAddress"}, {"transformationsData": [{"feature_label": "StoreCity", "transformation_label": "String Indexer"}], "feature": "StoreCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Amarillo", "max": "Wildorado", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StoreCity"}, {"transformationsData": [{"feature_label": "StoreState", "transformation_label": "String Indexer"}], "feature": "StoreState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "KS", "max": "TX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StoreState"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "73084.52", "stddev": "13365.53", "min": "43232", "max": "89506", "missing": "0"}, "updatedLabel": "StoreZip"}, {"transformationsData": [{"feature_label": "StoreCountry", "transformation_label": "String Indexer"}], "feature": "StoreCountry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "United States", "max": "United States", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StoreCountry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "32.27", "stddev": "2.27", "min": "29.3052293", "max": "43.98", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-99.13", "stddev": "10.02", "min": "-119.88658229999999", "max": "-82.8472342", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreLon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ProductAvailability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ProductAvailability"}, {"feature": "InternalOrderIDRef_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "128.23", "stddev": "103.32", "min": "0.0", "max": "340.0", "missing": "0"}, "updatedLabel": "InternalOrderIDRef_string..."}, {"feature": "OrderDate_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "161.04", "stddev": "96.17", "min": "0.0", "max": "244.0", "missing": "0"}, "updatedLabel": "OrderDate_stringindexer_t..."}, {"feature": "CustomerFirstName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "220.83", "stddev": "89.37", "min": "0.0", "max": "274.0", "missing": "0"}, "updatedLabel": "CustomerFirstName_stringi..."}, {"feature": "CustomerLastName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "88.52", "stddev": "83.35", "min": "0.0", "max": "276.0", "missing": "0"}, "updatedLabel": "CustomerLastName_stringin..."}, {"feature": "CustomerPhone_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "85.21", "stddev": "81.47", "min": "0.0", "max": "270.0", "missing": "0"}, "updatedLabel": "CustomerPhone_stringindex..."}, {"feature": "ProductName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "31.71", "stddev": "15.89", "min": "0.0", "max": "56.0", "missing": "0"}, "updatedLabel": "ProductName_stringindexer..."}, {"feature": "CustomerAddress_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "129.29", "stddev": "101.93", "min": "0.0", "max": "339.0", "missing": "0"}, "updatedLabel": "CustomerAddress_stringind..."}, {"feature": "CustomerCity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "39.14", "stddev": "38.98", "min": "0.0", "max": "149.0", "missing": "0"}, "updatedLabel": "CustomerCity_stringindexe..."}, {"feature": "CustomerState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.14", "stddev": "0.51", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "CustomerState_stringindex..."}, {"feature": "CustomerCountry_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "CustomerCountry_stringind..."}, {"feature": "ShippingMethod_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.35", "stddev": "1.35", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "ShippingMethod_stringinde..."}, {"feature": "Name_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "16.32", "stddev": "14.54", "min": "0.0", "max": "53.0", "missing": "0"}, "updatedLabel": "Name_stringindexer_transf..."}, {"feature": "SKU_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.99", "stddev": "0.1", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "SKU_binarizer_transform"}, {"feature": "RetailerItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "22.73", "stddev": "22.54", "min": "0.0", "max": "94.0", "missing": "0"}, "updatedLabel": "RetailerItemId_stringinde..."}, {"feature": "DimensionsUnit_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "DimensionsUnit_stringinde..."}, {"feature": "QuantityUnit_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "QuantityUnit_stringindexe..."}, {"feature": "weight_unit_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "weight_unit_stringindexer..."}, {"feature": "LocationType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "LocationType_stringindexe..."}, {"feature": "StoreRefId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "7.93", "stddev": "8.65", "min": "0.0", "max": "35.0", "missing": "0"}, "updatedLabel": "StoreRefId_stringindexer_..."}, {"feature": "StoreAddress_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "25.68", "stddev": "10.32", "min": "0.0", "max": "37.0", "missing": "0"}, "updatedLabel": "StoreAddress_stringindexe..."}, {"feature": "StoreCity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "6.33", "stddev": "6.13", "min": "0.0", "max": "28.0", "missing": "0"}, "updatedLabel": "StoreCity_stringindexer_t..."}, {"feature": "StoreState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.07", "stddev": "0.4", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "StoreState_stringindexer_..."}, {"feature": "StoreCountry_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "StoreCountry_stringindexe..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 1609211routingmodelHooks.ipynb
try:
	#mlPreExecutionHook()

	kmean = Clustering.run(autofe,spark,json.dumps( {"autoClustering": 0, "originalfile": "/FileStore/platform/sampleData/614346e76636f7335e83dfbc/part-00000-804e13f7-fb19-4837-8911-68528aaa69de-c000.csv", "model": "K-Means", "TrainingPercent": 100, "totalClusters": 70}))

	#mlPostExecutionHook(kmean)

except Exception as ex: 
	logging.error(ex)
